In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [2]:
# TODO: FILL IN PARAMETERS
# Model Parameter
model = "resnet34"
model_short = "resnet"
MODEL_MODULE = resnet34
# Dataset parameter
dataset = "cifar10"
dataset_short = "cifar"
DATASET_ENUM = DatasetClassN.Cifar

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 4

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [3]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [4]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [5]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=True)

NON Members
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 15000
Batchsize: 1
Inputs:tensor([[[[0.9529, 0.9412, 0.9412,  ..., 0.9412, 0.9490, 0.9490], [0.9451, 0.9255, 0.9255,  ..., 0.9569, 0.9647, 0.9569], [0.9333, 0.9176, 0.9176,  ..., 0.9569, 0.9529, 0.9569], ..., [0.6471, 0.5882, 0.5373,  ..., 0.6078, 0.6980, 0.7255], [0.4784, 0.4431, 0.4275,  ..., 0.4549, 0.5176, 0.5255], [0.5804, 0.5922, 0.6353,  ..., 0.5451, 0.5725, 0.5608]],[[0.9333, 0.9216, 0.9216,  ..., 0.9373, 0.9412, 0.9490], [0.9294, 0.9176, 0.9216,  ..., 0.9373, 0.9490, 0.9529], [0.9255, 0.9098, 0.9137,  ..., 0.9333, 0.9451, 0.9529], ..., [0.5608, 0.5137, 0.4706,  ..., 0.5255, 0.6078, 0.6314], [0.4196, 0.3922, 0.3725,  ..., 0.4118, 0.4627, 0.4627], [0.4863, 0.4902, 0.5294,  ..., 0.4784, 0.5098, 0.4980]],[[0.9216, 0.9098, 0.9137,  ..., 0.9294, 0.9373, 0.9490], [0.9137, 0.9020, 0.9059,  ..., 0.9216, 0.9373, 0.9529], [0.9098, 0.8941, 0.8980,  ..., 0.9216, 0.9294, 0.9451], ..., [0

In [6]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[-1.6158e-01,  6.8506e-01,  2.8033e-01]],  [[-2.6899e-01,  8.5374e-01, -1.1105e+00]],  [[-3.7050e-01,  6.4684e-02,  1.4932e+00]],  [[-5.0096e-01,  1.6212e+00,  2.7287e+00]],  [[-1.3435e+00, -4.9001e-01,  4.9706e-01]],  [[ 6.6649e-01, -8.5452e-01, -1.8427e+00]],  [[-1.1844e+00, -7.2732e-01, -1.3087e-01]],  [[-1.4677e+00, -3.5614e-01,  9.6374e-01]],  [[ 6.6832e-01,  3.5739e+00, -1.1785e+00]],  [[ 3.4454e+00, -1.0849e+00, -2.2010e+00]],  [[-2.4279e-01, -7.5349e-01, -4.4213e-01]],  [[-2.9250e-01,  4.3856e-01, -3.7936e-01]],  [[-3.0606e-02, -1.0897e+00, -5.4802e-01]],  [[ 1.9951e-01, -1.2578e+00, -4.4383e-01]],  [[ 1.4567e+00, -1.8961e+00, -1.7313e+00]],  [[-5.3257e-01,  2.1446e+00,  3.6775e-01]],  [[-8.5169e-01, -3.1222e-03,  8.9869e-01]],  [[-2.4089e-02,  4.8137e-01,  1.9204e-01]],  [[ 2.2333e+00, -1.5313e+00, -1.8384e+00]],  [[ 5.8765e-01, -1.7059e-01, -1.3197e-01]],  [[-6.9500e-01, -1.8974e-01,  1.0594e+00]],  [[-1.1885e+00, -2.0866e-01,  3.8878e-01]],  [[ 1.8731e-01, -8.8145e

### Attack Model

Simple Model

In [7]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [8]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [9]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [10]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE, test_dataset=False, standardize=True)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.3373, 0.3765, 0.3843,  ..., 0.2078, 0.2118, 0.2235], [0.4392, 0.5412, 0.5059,  ..., 0.2510, 0.2549, 0.2235], [0.4275, 0.4510, 0.4471,  ..., 0.2627, 0.2549, 0.2314], ..., [0.5373, 0.5647, 0.5529,  ..., 0.3490, 0.2745, 0.1765], [0.4157, 0.4353, 0.4549,  ..., 0.1608, 0.0941, 0.0745], [0.3882, 0.4000, 0.3961,  ..., 0.0627, 0.0627, 0.0627]],[[0.3765, 0.4314, 0.4314,  ..., 0.3412, 0.3333, 0.3490], [0.5529, 0.6784, 0.6275,  ..., 0.4039, 0.4157, 0.3804], [0.5059, 0.5569, 0.5765,  ..., 0.4235, 0.4118, 0.3843], ..., [0.6627, 0.7059, 0.7059,  ..., 0.3843, 0.2902, 0.1843], [0.4824, 0.5137, 0.5373,  ..., 0.1843, 0.1098, 0.0824], [0.4431, 0.4627, 0.4627,  ..., 0.0745, 0.0745, 0.0745]],[[0.1882, 0.2157, 0.2549,  ..., 0.4980, 0.4941, 0.5176], [0.1882, 0.2941, 0.2745,  ..., 0.5647, 0.5882, 0.5490], [0.3804, 0.2941, 0.2353,  ..., 0.5843, 0.5804, 0.5569], ..., [0.7765, 0.8314,

Attack Evaluation DL Sample

In [11]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

KeyboardInterrupt: 

### Train Attack model

In [ ]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Attack model state dictionary already available, loading into input model from attack_models/attack_resnet_cifar.pth!


### Evaluation

In [ ]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 28.56
